In [1]:
import random
from simanneal import Annealer

load("./consistency_conditions_scores.sage")


def fitness(individual):
    d = len(individual[0].tuple())
    P = SymmetricGroup(d)
    sigmaB = P(individual[0])
    sigmaW = P(individual[1])
    
    return 6 - (PT2_Score(sigmaB, sigmaW, d) + PT3a_Score(sigmaB, sigmaW, d) + PT3b_Score(sigmaB, sigmaW, d) + PT5_Score(sigmaB, sigmaW, d) + CONS1_Score(sigmaB, sigmaW, d) + CONS2_Score(sigmaB, sigmaW, d))


class ConsistentTripleFinder(Annealer):
    
    def __init__(self, state):
        super(ConsistentTripleFinder, self).__init__(state)  
         
    def move(self):
        initial_energy = self.energy()
        
        d = len(self.state[0].tuple())
        P = SymmetricGroup(d)
        
        sigmaB = []
        sigmaW = []
        
        num_cicles_B = len(self.state[0].cycle_tuples(singletons = True))
        num_cicles_W = len(self.state[1].cycle_tuples(singletons = True))
        
        for i in range(num_cicles_B):
            sigmaB.append(list(self.state[0].cycle_tuples(singletons = True)[i]))
        
        for i in range(num_cicles_W):
            sigmaW.append(list(self.state[1].cycle_tuples(singletons = True)[i]))
        
        # choose two random cicles in B and W
        c_i_B = random.randint(0, num_cicles_B - 1)
        c_j_B = random.randint(0, num_cicles_B - 1)
        c_i_W = random.randint(0, num_cicles_W - 1)
        c_j_W = random.randint(0, num_cicles_W - 1)
        
        # choose random elements in the cicles
        i_B = random.randint(0, len(self.state[0].cycle_tuples(singletons = True)[c_i_B]) - 1)
        j_B = random.randint(0, len(self.state[0].cycle_tuples(singletons = True)[c_j_B]) - 1)
        i_W = random.randint(0, len(self.state[1].cycle_tuples(singletons = True)[c_i_W]) - 1)
        j_W = random.randint(0, len(self.state[1].cycle_tuples(singletons = True)[c_j_W]) - 1)
        
        if random.random() < 0.5:
            sigmaB[c_i_B][i_B], sigmaB[c_j_B][j_B] = sigmaB[c_j_B][j_B], sigmaB[c_i_B][i_B]
        else:
            sigmaW[c_i_W][i_W], sigmaW[c_j_W][j_W] = sigmaW[c_j_W][j_W], sigmaW[c_i_W][i_W]
        
        sigmaB_t = [tuple(cycle) for cycle in sigmaB]
        sigmaW_t = [tuple(cycle) for cycle in sigmaW]
        
        self.state = [P(sigmaB_t), P(sigmaW_t)]
        
        E = self.energy()
       
       
        return E - initial_energy    

    def energy(self):
        return fitness(self.state)
    
    

In [ ]:
# Number of quantum fields
d = 26
P = SymmetricGroup(d)

# Initial seeds
# Observe the initial cycle-type of the seeds
ini_seed1 = P([(1,2,3),(4,5,6),(7,8,9),(10,11,12),(13,14,15),(16,17,18),(19,20,21,22),(23,24,25,26)])
ini_seed2 = P([(1,2,3),(4,5,6),(7,8,9),(10,11,12),(13,14,15),(16,17,18),(19,20,21,22),(23,24,25,26)])

init_state = [ini_seed1, ini_seed2]

sol = ConsistentTripleFinder(init_state)

# Increase time for more iterations
sol.set_schedule(sol.auto(minutes=1))  
sol.copy_strategy = "deepcopy"
state, e = sol.anneal()

print("------")
print(state)
print(e)


 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.72000          2.72    96.35%    45.85%     0:25:44     0:38:36

In [10]:
# Check the consistency of our new example

load("./consistency_conditions_binary.sage")

d = 26
P = SymmetricGroup(d)
sigmaB = P([(1,3,21,14),(2,4,8),(5,19,24,26),(6,7,25),(9,17,11),(10,23,18),(12,22,16),(13,15,20)])
sigmaW = P([(1,10,2,6),(3,25,16),(4,12,24),(5,20,23,14),(7,13,26),(8,17,15),(9,22,18),(11,19,21)])

PT2(sigmaB, sigmaW, d), PT3a(sigmaB, sigmaW, d), PT3b(sigmaB, sigmaW, d), PT5(sigmaB, sigmaW, d), CONS1(sigmaB, sigmaW, d), CONS2(sigmaB, sigmaW, d)

(True, True, True, True, True, True)